# Canada fire chronosequence
- Height is from living trees. 
- Distinction between nutrient poor and nutrient rich sites, but I discarded the distinction BC when taking a weighted mean of fall rates with the density of dead trees / site, there is only 1 year difference between estimates. 
- Also weighing the estimates by dead density, mean BA, mean Vol does not result in a measurable difference. Reporting ntree here, BA and Vol differ by 1 year both from ntree estimate.

![title](snag_fall_rates/Metsaranta2008DendrochronologicalCanada_tab3.png)

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as opt
from scipy.interpolate import interp1d
import sys

sys.path.append('/Users/antje/Update_PIK/')

from classes import snag_MTF_func as sg
from classes import boundaries as bd
from classes import functions as fcy

plot_setup = bd.plot_setup()
params = {'mathtext.default':'regular'}
plt.rcParams.update(params)

inch_to_cm = 2.54

%matplotlib inline
%config Completer.use_jedi = False

In [3]:
site_names           = ['Prince Albert', 'Candle Lake', 'Flin Flon', 'Jenpeg', 'Thompson']
site_names_poor      = [i + '- poor' for i in site_names]
site_names_rich      = [i + '- rich' for i in site_names]
site_names_all       = [None]*(len(site_names_poor)+len(site_names_rich))
site_names_all[::2]  = site_names_rich
site_names_all[1::2] = site_names_poor
site_names_all

['Prince Albert- rich',
 'Prince Albert- poor',
 'Candle Lake- rich',
 'Candle Lake- poor',
 'Flin Flon- rich',
 'Flin Flon- poor',
 'Jenpeg- rich',
 'Jenpeg- poor',
 'Thompson- rich',
 'Thompson- poor']

In [4]:
##############
#   Import   #
##############

save = False # put to FALSE while WIP!

reference = 'Metsaranta2008DendrochronologicalCanada'

species  = [           
            ['Jack pine', 'Pinus banksiana']
           ]

 
country              = 'Canada'
region               = 'Saskatchewan'
biome                = 'Boreal'
site_names           = ['Prince Albert', 'Candle Lake', 'Flin Flon', 'Jenpeg', 'Thompson']
site_names_poor      = [i + '- poor' for i in site_names]
site_names_rich      = [i + '- rich' for i in site_names]
site_names_all       = [None]*(len(site_names_poor)+len(site_names_rich))
site_names_all[::2]  = site_names_rich
site_names_all[1::2] = site_names_poor
mean_tC              = np.nan      # [degree C]
mean_precip_mm       = np.nan      # [mm]
y_coords             = [53.277095765986246, # Prince Albert -> Boreal Plains ecozone
                        53.831585653175594, # Candle Lake   -> "" 
                        54.830860656055854, # Flin Flon     -> Boreal shield ecozone
                        54.64062521100422 , # Jenpeg        -> ""
                        55.778676150241054  # Thompson      -> ""
                       ]
y_coords_all         = np.repeat(y_coords, 2)  
x_coords             = [-105.75388715328937, 
                        -105.1913405399521 , 
                        -101.87189632630448, 
                        -98.04416921893932 , 
                        -97.83933156686099 
                       ]
x_coords_all         = np.repeat(x_coords, 2)

tlag               = np.nan      # n years until sangfall starts
mean_age           = [74,76,84,84,78,74,80,71,71,69]      # Table 1: DBH age (year of origin)
elevation          = np.nan
min_height         = np.nan      # [m]
min_diam           = np.nan      # [cm]
max_diam           = np.nan

DBH_classes        = np.nan
DBH_classes_mean   = np.nan

mean_DBH_all       = np.array([14.2,13.7,17.4,12.7,14.9,12.4,17.8,14.7,15.9,9.4]) # Table 1
mean_DBH_poor      = np.mean(mean_DBH_all[1::2])
mean_DBH_rich      = np.mean(mean_DBH_all[0::2])
mean_DBH           = [mean_DBH_poor, mean_DBH_rich]

mean_height_all    = np.array([14.6,12.7,15.9,13,14.9,12.2,15.6,13.4,13.7,8]) # Table 1
mean_height_poor   = np.mean(mean_height_all[1::2])
mean_height_rich   = np.mean(mean_height_all[0::2])
mean_height        = [mean_height_poor, mean_height_rich]

dead_density       = np.array([3900-2200, 
                               3300-2000,
                               1767-1267,
                               2556-1656,
                               5300-1400,
                               6300-1500,
                               3000-1200,
                               3700-1200,
                               1900-1367,
                               3222-2089]) # table 1: dead+live density-live density / ha
dead_density_poor  = np.array(dead_density[1::2])
dead_density_rich  = np.array(dead_density[0::2])

tree_density       = np.array([3900, 
                               3300,
                               1767,
                               2556,
                               5300,
                               6300,
                               3000,
                               3700,
                               1900,
                               3222]) # live+dead density # ha-1 table 1

speciesDOM         = 'Pinus banksiana' # scientific only
speciesDOM_prc     = np.nan # dominant tree species percentage of total

species_ntrees     = np.array([11,9,46,68,12,20,5,8,50,78])

n_plots            = 1
plot_size          = [0.09,0.09,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01] # ha
management         = 'Unmanaged; fire origin'
mortality_cause    = np.nan
    
species_abundance_prc = np.nan
age_classes           = np.nan
tree_share_to_snag    = np.nan

# Load data types
dt = pd.read_excel("snag_fall_rates/201210_data_types_MTF_literature.xlsx", index_col=None, header=[0,1])['Data types']
method_snag_field_measurement      = dt.columns[(dt == reference).any()].item()

ntree_Total                        = species_ntrees.sum()
Remeasurement_interval_avg         = 1     # years
survey_duration_total              = 70     # years
survey_start_yr                    = 1935
survey_end_yr                      = 1995
TSD_determination_method           = 'Dendrochronology'   #cf. Method_snagfall_measurement -> Dendrochronology, Chronosequence, None if mean fall /year
Model_type_fracRemain_snag         = 'Single exponential model, maximum likelihood estimate for k following Gore et al. (1985)'# ‘negative exponential…
Model_type_fracRemain_snag_sigCov  = np.nan # significant (p<0.05) co-variats
Model_type_fracRemain_snag_nsigCov = np.nan # not significant co-variats




# Path to data
path = '../data/snag_fall_rates/'

#chambers = pd.read_csv(path+reference+'_fig1.csv', 
#            header=[0,1]
#                     )

metsaranta_arr = [0.02,0.014,0.015,0.016,0.04,0.034,0.038,0.05,0.018,0.034] # Snag fall rate as trees / year
#metsaranta_arr = [sg.annualFall2k(f) for f in metsaranta_arr] # this is not needed because 1/k = MTF, and fall rate equals the mean standing time
nutrient_status = np.tile(np.array(['rich', 'poor']),5)
metsa_poor = metsaranta_arr[1::2]
metsa_rich = metsaranta_arr[0::2]
metsa_rich_k = np.mean(metsa_rich)
metsa_poor_k = np.mean(metsa_poor)

In [5]:
k_wghtd_poor = 1/(np.sum(metsa_poor_k * dead_density_poor)/np.sum(dead_density_poor))
k_wghtd_rich = 1/(np.sum(metsa_rich_k * dead_density_rich)/np.sum(dead_density_rich))


print('k weighted ø poor: ', np.round(k_wghtd_poor))
print('k weighted ø rich: ', np.round(k_wghtd_rich))
print()
print()
print('k mean ø poor: ',np.round(1/metsa_poor_k))
print('k mean ø rich: ',np.round(1/metsa_rich_k))

k_all_ntree = 1 / (np.sum(metsaranta_arr*dead_density) /np.sum(dead_density))

print()
print('k wght ntree ø: ', np.round(k_all_ntree))


k weighted ø poor:  34.0
k weighted ø rich:  38.0


k mean ø poor:  34.0
k mean ø rich:  38.0

k wght ntree ø:  30.0


In [6]:
1/np.array(metsaranta_arr)

array([50.        , 71.42857143, 66.66666667, 62.5       , 25.        ,
       29.41176471, 26.31578947, 20.        , 55.55555556, 29.41176471])

In [7]:
#------------------------#
#   Store MTF directly   #
#------------------------#
MTF_all                            = 1/np.array(metsaranta_arr)
DBH_quadmean_site                  = np.nan
species_quadmean_dbh               = np.nan
N_species_contributing_to_site_MTF = 1
MTF_method_notes                   = 'Table 3'
MTF_method                         = 'fall rate'
category                           = 'count'
species_list                       = species[0]
species_ntree                      = species_ntrees
species_prc                        = speciesDOM_prc
n_dbh_class                        = np.nan

for i in range(len(site_names_all)):
    DBH_mean_site = mean_DBH_all[i]
    species_mean_dbh = mean_DBH_all[i]
    sg.store_MTF_site_from_k(MTF_all[i],
                             category,
                             MTF_method,
                             reference,
                             site_names_all[i],
                             speciesDOM,
                             speciesDOM_prc,
                             DBH_mean_site,
                             DBH_quadmean_site,
                             N_species_contributing_to_site_MTF,
                             MTF_method_notes)
    sg.store_MTF_species_from_k(reference,
                                MTF_all[i],
                                MTF_method,
                                site_names_all[i],
                                category,
                                species_list,
                                species_ntree[i],
                                species_mean_dbh,
                                species_quadmean_dbh,
                                species_prc,
                                n_dbh_class)

removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine
removing the old ENTRY
removing the old ENTRY count Jack pine


In [8]:
for i in range(len(site_names_all)):
    sg.store_single_loc(country,
                        region,
                        biome,
                        site_names_all[i],
                        y_coords_all[i],
                        x_coords_all[i],
                        min_height,
                        min_diam,
                        max_diam,
                        species[0][1],
                        speciesDOM_prc,
                        mortality_cause,
                        management,
                        mean_age[i],
                        method_snag_field_measurement,
                        tree_share_to_snag,
                        n_plots,
                        plot_size[i],
                        species_ntrees[i],
                        tree_density,
                        Remeasurement_interval_avg,
                        survey_duration_total,
                        survey_start_yr,
                        survey_end_yr,
                        TSD_determination_method,
                        Model_type_fracRemain_snag,
                        Model_type_fracRemain_snag_sigCov,
                        Model_type_fracRemain_snag_nsigCov,
                        elevation,
                        mean_tC,
                        mean_precip_mm,
                        reference)

removing the old ENTRY
removing the old ENTRY
removing the old ENTRY
removing the old ENTRY
removing the old ENTRY
removing the old ENTRY
removing the old ENTRY
removing the old ENTRY
removing the old ENTRY
removing the old ENTRY


/Users/antje/Update_PIK/classes/snag_MTF_func.py:1762: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sites = sites.append(next_row,ignore_index=True)
/Users/antje/Update_PIK/classes/snag_MTF_func.py:1762: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sites = sites.append(next_row,ignore_index=True)
/Users/antje/Update_PIK/classes/snag_MTF_func.py:1762: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sites = sites.append(next_row,ignore_index=True)
/Users/antje/Update_PIK/classes/snag_MTF_func.py:1762: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sites = sites.append(next_row,ignore_index=True)
/Users/antje/Update_PIK/classes/snag_MTF_fun